In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
pd.options.display.max_rows = 250


import plotly.express as px
import seaborn as sns


In [2]:
files = glob.glob('../mens-march-mania-2022/MDataFiles_Stage2/*')
[print(i,'\m/',f) for i,f in enumerate(files)];

0 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneyDetailedResults.csv
1 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneyCompactResults.csv
2 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MSeasons.csv
3 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MMasseyOrdinals_thruDay128.csv
4 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonDetailedResults.csv
5 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneySlots.csv
6 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MGameCities.csv
7 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MConferenceTourneyGames.csv
8 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/Cities.csv
9 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonCompactResults.csv
10 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneySeedRoundSlots.csv
11 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MTeamConferences.csv
12 \m/ ../mens-march-mania-2022/MDataFiles_Stage2/MSampleSubmissionStage2.csv
13 \m/ ../mens-march-mania-20

In [3]:
#1) Use ../mens-march-mania-2022/MDataFiles_Stage2/MTeams.csv to creat team df
teams = pd.read_csv(files[15],usecols=['TeamID','TeamName'])
#teams

# 2) NCAA Basketball Season Data (../mens-march-mania-2022/MDataFiles_Stage2/MSeasons.csv)
seasons = pd.read_csv(files[2])
#seasons

# 3) Seeds in tournament
seeds = pd.read_csv(files[16])
#seeds.tail()

# 4) Merge Teams and Tourney Seed DF's
seeds = pd.merge(seeds, teams,on='TeamID')
seeds.tail()

,Season,Seed,TeamID,TeamName
2417,2019,Z16,1205,Gardner Webb
2418,2021,X15,1213,Grand Canyon
2419,2021,Z16,1216,Hartford
2420,2022,Z14,1255,Longwood
2421,2022,Z16a,1136,Bryant


In [4]:
# 5) Separating regions and Seeds (Ex: making Seed Z16 go to 16)

seeds['Region'] = seeds['Seed'].apply(lambda x: x[0][:1])
seeds['Seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))

print(seeds.head())
print(seeds.shape)

   Season  Seed  TeamID    TeamName Region
0    1985     1    1207  Georgetown      W
1    1986     4    1207  Georgetown      X
2    1987     1    1207  Georgetown      X
3    1988     8    1207  Georgetown      W
4    1989     1    1207  Georgetown      W
(2422, 5)


In [5]:
seeds
#seeds.to_csv('GroupProj_Seeds_DF.csv', encoding='utf-8', index=False)

,Season,Seed,TeamID,TeamName,Region
0,1985,1,1207,Georgetown,W
1,1986,4,1207,Georgetown,X
2,1987,1,1207,Georgetown,X
3,1988,8,1207,Georgetown,W
4,1989,1,1207,Georgetown,W
...,...,...,...,...,...
2417,2019,16,1205,Gardner Webb,Z
2418,2021,15,1213,Grand Canyon,X
2419,2021,16,1216,Hartford,Z
2420,2022,14,1255,Longwood,Z


In [6]:
# 6) Use most updated ../mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonCompactResults file to retrieve season results
## 6 or 7 - 

SeasonResults = pd.read_csv(files[4])

#SeasonResults

winning_cols = ['Season', 'WTeamID', 'WLoc', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']
losing_cols = ['Season', 'LTeamID', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']
columns = ['Season', 'TeamID', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']

winning_stats = SeasonResults[winning_cols].groupby(['Season', 'WTeamID']).mean().reset_index()
winning_stats.columns = columns

losing_stats = SeasonResults[losing_cols].groupby(['Season', 'LTeamID']).mean().reset_index()
losing_stats.columns = columns

reg_season_stats = pd.concat([winning_stats, losing_stats]).groupby(['Season', 'TeamID']).mean().reset_index()
reg_season_stats['id'] = reg_season_stats.Season.astype(str) + reg_season_stats.TeamID.astype(str)


reg_season_stats.T
#reg_season_stats.to_csv('GroupProj_RegSeas_DF.csv', encoding='utf-8', index=False)

,0,1,2,3,4,5,6,7,8,9,...,6882,6883,6884,6885,6886,6887,6888,6889,6890,6891
Season,2003,2003,2003,2003,2003,2003,2003,2003,2003,2003,...,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022
TeamID,1102,1103,1104,1105,1106,1107,1108,1110,1111,1112,...,1463,1464,1465,1466,1467,1468,1469,1470,1471,1472
FGM,19.572917,27.25,23.548128,24.759398,23.517949,24.357143,25.088346,23.053571,28.18125,29.153333,...,25.123737,25.392857,23.7,24.142857,22.447115,25.206731,24.861111,22.596257,23.647727,27.082143
FGA,39.8125,55.835165,56.858289,61.691729,55.189744,56.880952,58.415414,53.607143,65.08125,68.92,...,58.176768,57.258929,54.966667,60.904762,52.870192,55.151442,59.083333,53.411765,52.948864,58.857143
FGM3,8.09375,5.445055,6.165775,8.071429,6.089744,7.952381,5.296992,8.696429,6.96875,6.873333,...,6.95202,7.660714,7.866667,7.552381,7.240385,7.790865,6.888889,5.435829,8.982955,11.007143
FGA3,20.916667,16.013736,19.593583,21.293233,17.528205,21.785714,16.176692,23.196429,22.55,23.12,...,21.75,22.294643,23.566667,23.904762,21.355769,23.158654,22.055556,16.874332,24.215909,29.842857
FTM,11.447917,19.153846,14.545455,16.597744,10.794872,10.714286,14.394737,11.183036,18.48125,16.42,...,13.573232,12.825893,13.933333,11.0,9.492788,9.057692,13.833333,13.820856,13.767045,11.782143
FTA,17.5625,26.032967,20.491979,23.067669,16.751282,14.309524,21.360902,16.379464,25.74375,23.973333,...,18.616162,16.90625,20.566667,14.709524,14.004808,12.074519,20.555556,18.414439,18.224432,14.746429
OR,4.135417,9.763736,13.582888,13.838346,12.317949,8.404762,12.93609,10.017857,13.2625,16.42,...,7.416667,8.75,8.6,8.97619,6.048077,6.161058,8.583333,7.526738,4.653409,5.742857
DR,17.135417,19.980769,23.251337,23.981203,24.133333,21.833333,23.419173,23.071429,24.36875,25.746667,...,25.161616,22.334821,24.533333,22.895238,19.28125,20.115385,24.805556,18.625668,22.588068,20.142857


In [7]:
# 7)
# Dropping NumOT -  (Overtime data can remove)
## Compliling RegularSeasonResultsGamelogs and Tourney Game data -
### Using pd.concat to arrange/merge dataframes -
####Add a column called Score Differential = WScore - LScore
######'GameID'

SeasonResults = pd.read_csv(files[9])

SeasonResults['GameID'] = np.arange(1,len(SeasonResults)+1)

SeasonResultsWin = SeasonResults[['Season','DayNum','GameID','WTeamID','WScore','LTeamID','LScore','WLoc']].copy()

SeasonResultsWin.columns = ['Season','DayNum','GameID','TeamID','Score','TeamID_OPP','Score_OPP','WLoc']
SeasonResultsWin['result'] = 1

SeasonResultsLose = SeasonResults[['Season','DayNum','GameID','WTeamID','WScore','LTeamID','LScore','WLoc']].copy()
SeasonResultsLose.columns = ['Season','DayNum','GameID','TeamID_OPP','Score_OPP','TeamID','Score','WLoc']
SeasonResultsLose['result'] = 0

SeasonResultsTeams = pd.concat([SeasonResultsWin,SeasonResultsLose],axis=0,sort=True).sort_values('GameID')
SeasonResultsTeams['WL'] = np.where(SeasonResultsTeams['Score']-SeasonResultsTeams['Score_OPP']>0,'W','L')

SeasonResultsTeams['Score_Diff'] = SeasonResultsTeams['Score'] - SeasonResultsTeams['Score_OPP']

SeasonResultsTeams.tail()

#SeasonResultsTeams.to_csv('GroupProj_RegSeasII_DF.csv', encoding='utf-8', index=False)

SeasonResultsTeams.tail()
# 352160 rows × 10 columns

,DayNum,GameID,Score,Score_OPP,Season,TeamID,TeamID_OPP,WLoc,result,WL,Score_Diff
176077,132,176078,64,62,2022,1350,1172,N,1,W,2
176078,132,176079,50,65,2022,1401,1397,N,0,L,-15
176078,132,176079,65,50,2022,1397,1401,N,1,W,15
176079,132,176080,66,64,2022,1463,1343,N,1,W,2
176079,132,176080,64,66,2022,1343,1463,N,0,L,-2


In [8]:
massey_ranks = pd.read_csv(files[3])

ranks_agg = massey_ranks.groupby(['Season', 'TeamID']).agg({'OrdinalRank': ['mean', 'min', 'max']})
ranks_agg.columns = ['_'.join(col) for col in ranks_agg.columns]

team_ranks = ranks_agg.reset_index()
team_ranks['id'] = team_ranks.Season.astype(str) + team_ranks.TeamID.astype(str)
team_ranks.tail()

#team_ranks.to_csv('Massey_Rankings03-22.csv', encoding='utf-8', index=False)

,Season,TeamID,OrdinalRank_mean,OrdinalRank_min,OrdinalRank_max,id
6897,2022,1468,206.520604,113,332,20221468
6898,2022,1469,271.579670,175,342,20221469
6899,2022,1470,220.846154,122,345,20221470
6900,2022,1471,238.221154,34,313,20221471
6901,2022,1472,285.106443,114,358,20221472


# 1 - Predict Tournament winning probability from regular seaon data

In [9]:
def is_winning(wteam, lteam):
    if wteam < lteam:
        return 1
    else:
        return 0
    
    
march_madness = pd.read_csv(files[1])
train = march_madness

train['winning'] = train.apply(lambda x: is_winning(x['WTeamID'], x['LTeamID']), axis=1)

train['team_a'] = train.Season.astype(str) + train.WTeamID.astype(str)

train['team_b'] = train.Season.astype(str) + train.LTeamID.astype(str)

train = train.drop(['WScore', 'LScore'], axis=1)
train = pd.merge(train, team_ranks, left_on='team_a', right_on='id').merge(team_ranks, left_on='team_b', right_on='id', suffixes=('_teama', '_teamb'))

train = train.drop(['Season_x', 'Season_y', 'id_teama', 'id_teamb', 'TeamID_teama', 'TeamID_teamb'], axis=1)
train = pd.merge(train, reg_season_stats, left_on='team_a', right_on='id').merge(reg_season_stats, left_on='team_b', right_on='id', suffixes=('_teama', '_teamb'))

train = train.drop(['Season_x', 'Season_y', 'team_a', 'team_b', 'TeamID_teama', 'TeamID_teamb', 'id_teama', 'id_teamb'], axis=1)
train = train.drop(['DayNum', 'WTeamID', 'LTeamID', 'NumOT', 'Season', 'WLoc'], axis=1)

print(train.shape)
train.head()

(1181, 33)


,winning,OrdinalRank_mean_teama,OrdinalRank_min_teama,OrdinalRank_max_teama,OrdinalRank_mean_teamb,OrdinalRank_min_teamb,OrdinalRank_max_teamb,FGM_teama,FGA_teama,FGM3_teama,...,FGA3_teamb,FTM_teamb,FTA_teamb,OR_teamb,DR_teamb,Ast_teamb,TO_teamb,Stl_teamb,Blk_teamb,PF_teamb
0,0,241.370000,128,315,236.217500,114,324,24.646635,56.891827,6.524038,...,18.750000,16.722222,27.194444,13.458333,24.111111,13.958333,14.958333,6.569444,2.277778,18.208333
1,1,3.676744,1,47,171.797500,101,265,29.153333,68.920000,6.873333,...,15.960526,12.278947,18.994737,12.902632,25.102632,13.542105,14.336842,6.776316,2.902632,16.181579
2,1,3.676744,1,47,43.222500,21,107,29.153333,68.920000,6.873333,...,19.410326,17.603261,24.190217,11.589674,24.483696,15.377717,14.328804,6.421196,3.673913,19.086957
3,1,3.676744,1,47,10.497674,1,113,29.153333,68.920000,6.873333,...,22.366162,16.921717,22.590909,11.896465,25.333333,16.439394,12.545455,6.957071,5.126263,16.618687
4,1,40.892500,10,145,42.000000,7,109,26.618687,57.199495,4.141414,...,20.228261,15.902174,24.289855,14.474638,24.992754,15.221014,13.376812,7.115942,5.105072,20.079710


In [10]:
from sklearn.model_selection import train_test_split

X = train.drop(['winning'], axis=1)
y = train.winning

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.71      0.67       126
           1       0.63      0.56      0.59       111

    accuracy                           0.64       237
   macro avg       0.64      0.63      0.63       237
weighted avg       0.64      0.64      0.64       237



In [12]:
def return_value(data, data_type=None):
    data = data.split('_')
    if data_type == 'Team A':
        return str(data[0]) + str(data[1])
    else:
        return str(data[0]) + str(data[2])


test = pd.read_csv(files[12])

test['team_a'] = test.apply(lambda x: return_value(x.ID, 'Team A'), axis=1)
test['team_b'] = test.apply(lambda x: return_value(x.ID), axis=1)
test = pd.merge(test, team_ranks, left_on='team_a', right_on='id').merge(team_ranks, left_on='team_b', right_on='id', suffixes=('_teama', '_teamb'))
test = pd.merge(test, reg_season_stats, left_on='team_a', right_on='id').merge(reg_season_stats, left_on='team_b', right_on='id', suffixes=('_teama', '_teamb'))
test = test.drop(['Season_teama', 'Season_teamb', 'team_a', 'team_b', 'TeamID_teama', 'TeamID_teamb', 'id_teama', 'id_teamb'], axis=1)

test.head()

/Users/apple/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/frame.py:9203: FutureWarning: Passing 'suffixes' which cause duplicate columns {'id_teama', 'TeamID_teama', 'Season_teama'} in the result is deprecated and will raise a MergeError in a future version.
  validate=validate,


,ID,Pred,OrdinalRank_mean_teama,OrdinalRank_min_teama,OrdinalRank_max_teama,OrdinalRank_mean_teamb,OrdinalRank_min_teamb,OrdinalRank_max_teamb,FGM_teama,FGA_teama,...,FGA3_teamb,FTM_teamb,FTA_teamb,OR_teamb,DR_teamb,Ast_teamb,TO_teamb,Stl_teamb,Blk_teamb,PF_teamb
0,2022_1103_1104,0.5,142.390110,51,327,17.380764,1,51,22.851010,52.699495,...,30.309717,15.255061,20.94332,11.684211,24.374494,14.514170,14.115385,6.987854,4.605263,18.840081
1,2022_1103_1112,0.5,142.390110,51,327,4.730323,1,51,22.851010,52.699495,...,23.005376,14.145161,19.50000,10.693548,26.989247,17.091398,14.172043,5.634409,4.784946,17.462366
2,2022_1104_1112,0.5,17.380764,1,51,4.730323,1,51,27.370445,62.850202,...,23.005376,14.145161,19.50000,10.693548,26.989247,17.091398,14.172043,5.634409,4.784946,17.462366
3,2022_1103_1116,0.5,142.390110,51,327,33.667105,3,147,22.851010,52.699495,...,21.775000,16.817500,22.42750,9.630000,24.002500,13.847500,13.035000,7.480000,3.950000,17.792500
4,2022_1104_1116,0.5,17.380764,1,51,33.667105,3,147,27.370445,62.850202,...,21.775000,16.817500,22.42750,9.630000,24.002500,13.847500,13.035000,7.480000,3.950000,17.792500


In [13]:
X = test.drop(['ID', 'Pred'], axis=1)

test['Pred'] = model.predict_proba(X)[:, 1]
test = test[['ID', 'Pred']]
test.head()

,ID,Pred
0,2022_1103_1104,0.42
1,2022_1103_1112,0.39
2,2022_1104_1112,0.62
3,2022_1103_1116,0.53
4,2022_1104_1116,0.61


In [14]:
test.to_csv('prediction_scores.csv', index=False)

In [15]:
pd.read_csv('prediction_scores.csv')

,ID,Pred
0,2022_1103_1104,0.42
1,2022_1103_1112,0.39
2,2022_1104_1112,0.62
3,2022_1103_1116,0.53
4,2022_1104_1116,0.61
...,...,...
2273,2022_1437_1463,0.38
2274,2022_1439_1463,0.47
2275,2022_1458_1463,0.45
2276,2022_1460_1463,0.35


In [16]:
#8) Home vs Away all time- 
#Could do something with this but often not considered for tournament is on neutral court

# ax = sns.countplot(x=SeasonResults['WLoc'])
# ax.set_title("Win Locations")
# ax.set_xlabel("Location")
# ax.set_ylabel("Frequency");

In [17]:
# 9) Extract tourney data

TourneyCompactResults = pd.read_csv(files[1])
TourneyCompactResults.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
2312,2021,148,1211,85,1425,66,N,0
2313,2021,148,1417,51,1276,49,N,0
2314,2021,152,1124,78,1222,59,N,0
2315,2021,152,1211,93,1417,90,N,1
2316,2021,154,1124,86,1211,70,N,0


In [18]:
# 10) Games played each year
games_played = TourneyCompactResults.groupby('Season')['DayNum'].count().to_frame().merge(SeasonResultsTeams.groupby('Season')['DayNum'].count().to_frame(),on='Season')
games_played.rename(columns={"DayNum_x": "Tournament Games", "DayNum_y": "Regular season games"})

,Tournament Games,Regular season games
Season,,
1985,63,7474
1986,63,7566
1987,63,7830
1988,63,7910
1989,63,8074
1990,63,8090
1991,63,8246
1992,63,8254
1993,63,7964


In [19]:
# 11) team-level box scores for all NCAA tournaments, in the 2003-2021 season data
# Add score differential to tourney_detailed_results

tourney_detailed_results = pd.read_csv(files[0])

#tourney_detailed_results['Score_Diff'] = TourneyCompactResults['WScore'] - TourneyCompactResults['LScore']
tourney_detailed_results.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
1176,2021,148,1211,85,1425,66,N,0,33,66,...,15,14,19,7,20,9,9,7,0,13
1177,2021,148,1417,51,1276,49,N,0,21,54,...,11,6,11,8,24,12,14,5,3,11
1178,2021,152,1124,78,1222,59,N,0,29,55,...,19,11,16,13,12,10,10,4,5,10
1179,2021,152,1211,93,1417,90,N,1,37,63,...,17,14,21,7,24,21,9,4,1,16
1180,2021,154,1124,86,1211,70,N,0,30,67,...,17,15,21,1,16,16,14,4,3,17


In [20]:
tourney_detailed_results.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [21]:
## 12) We need a way to compile and merge game stats into a Winning Team ID(WTEAMID) & Losing Team ID(LTEAMID) etc.

games_stats = []

for row in tourney_detailed_results.to_dict('records'):
    game = {}
    game['Season'] =  row['Season']
    game['DayNum'] = row['DayNum']
    game['TeamID'] = row['WTeamID']
    game['OpponentID'] = row['LTeamID']
    game['FGM'] = row['WFGM']
    game['Loc'] = row['WLoc']
    game['Won'] = 1
    game['Score'] = row['WScore']
    game['FGA'] = row['WFGA']
    game['FGM3'] = row['WFGM3']
    game['FGA3'] = row['WFGA3']
    game['FTM'] = row['WFTM']
    game['FTA'] = row['WFTA']
    game['OR'] = row['WOR']
    game['DR'] = row['WDR']
    game['AST'] = row['WAst']
    game['TO'] = row['WTO']
    game['STL'] = row['WStl']
    game['BLK'] = row['WBlk']
    game['PF'] = row['WPF']
    #game['Score_Diff'] = row['Score_Diff']
    games_stats.append(game)
    
    game = {}
    game['Season'] = row['Season']
    game['DayNum'] = row['DayNum']
    game['TeamID'] = row['LTeamID']
    game['OpponentID'] = row['WTeamID']
    game['FGM'] = row['LFGM']
    game['Loc'] = row['WLoc']
    game['Won']= 0
    game['Score'] = row['LScore']
    game['FGA'] = row['LFGA']
    game['FGM3'] = row['LFGM3']
    game['FGA3'] = row['LFGA3']
    game['FTM'] = row['LFTM']
    game['FTA'] = row['LFTA']
    game['OR'] = row['LOR']
    game['DR'] = row['LDR']
    game['AST'] = row['LAst']
    game['TO'] = row['LTO']
    game['STL'] = row['LStl']
    game['BLK'] = row['LBlk']
    game['PF'] = row['LPF']
    #game['Score_Diff'] = row['Score_Diff']
    games_stats.append(game)

In [22]:
tournament = pd.DataFrame(games_stats)
tournament.tail(25)

,Season,DayNum,TeamID,OpponentID,FGM,Loc,Won,Score,FGA,FGM3,FGA3,FTM,FTA,OR,DR,AST,TO,STL,BLK,PF
2337,2021,145,1393,1222,14,N,0,46,50,5,23,13,17,4,19,5,11,3,4,17
2338,2021,145,1333,1260,21,N,1,65,51,5,13,18,20,8,22,13,7,4,2,16
2339,2021,145,1260,1333,18,N,0,58,54,5,23,17,24,11,20,14,8,4,4,20
2340,2021,146,1211,1166,34,N,1,83,57,9,24,6,10,6,28,23,16,5,2,12
2341,2021,146,1166,1211,28,N,0,65,69,5,23,4,8,8,17,11,10,9,0,12
2342,2021,146,1276,1199,29,N,1,76,59,3,11,15,23,9,25,19,7,4,3,10
2343,2021,146,1199,1276,24,N,0,58,60,5,20,5,6,7,21,10,14,6,5,22
2344,2021,146,1417,1104,29,N,1,88,74,10,29,20,25,12,26,12,7,8,7,21
2345,2021,146,1104,1417,30,N,0,78,69,7,28,11,25,11,30,9,14,6,5,23
2346,2021,146,1425,1332,31,N,1,82,54,10,17,10,12,8,20,15,13,5,4,11


In [23]:
tourny_df = pd.merge(tournament , seeds, on= ['Season','TeamID'])
tourny_df.rename(columns={'Seed': 'Team_Seed'}, inplace=True)
tourny_df

,Season,DayNum,TeamID,OpponentID,FGM,Loc,Won,Score,FGA,FGM3,...,OR,DR,AST,TO,STL,BLK,PF,Team_Seed,TeamName,Region
0,2003,134,1421,1411,32,N,1,92,69,11,...,14,30,17,12,5,3,22,16,UNC Asheville,X
1,2003,137,1421,1400,23,N,0,61,62,5,...,13,16,5,11,9,3,16,16,UNC Asheville,X
2,2003,134,1411,1421,29,N,0,84,67,12,...,17,28,16,15,5,0,22,16,TX Southern,X
3,2003,136,1112,1436,31,N,1,80,66,7,...,11,36,22,16,10,7,8,1,Arizona,Z
4,2003,138,1112,1211,34,N,1,96,74,7,...,18,29,18,9,7,4,20,1,Arizona,Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,2021,140,1425,1242,32,N,1,85,56,11,...,6,32,18,13,3,4,14,6,USC,X
2358,2021,146,1425,1332,31,N,1,82,54,10,...,8,20,15,13,5,4,11,6,USC,X
2359,2021,148,1425,1211,24,N,0,66,62,4,...,7,20,9,9,7,0,13,6,USC,X
2360,2021,140,1332,1234,38,N,1,95,68,11,...,7,23,25,10,7,6,13,7,Oregon,X


In [24]:
tournament_df = pd.merge(tourny_df , seeds.rename(columns={'TeamID':'OpponentID'}), on= ['Season','OpponentID'])
tournament_df .rename(columns={'Seed': 'OpponentSeed',
                                'TeamName_x':'Team',
                                'TeamName_y':'Opponents',
                                 'Region_x':'Team_Region',
                                 'Region_y':'Opponent_Region'}, inplace=True)


## Tournament_df now has winning/ losing unique id, Seed Place, Regions
tournament_df

,Season,DayNum,TeamID,OpponentID,FGM,Loc,Won,Score,FGA,FGM3,...,TO,STL,BLK,PF,Team_Seed,Team,Team_Region,OpponentSeed,Opponents,Opponent_Region
0,2003,134,1421,1411,32,N,1,92,69,11,...,12,5,3,22,16,UNC Asheville,X,16,TX Southern,X
1,2003,137,1421,1400,23,N,0,61,62,5,...,11,9,3,16,16,UNC Asheville,X,1,Texas,X
2,2003,144,1163,1400,30,N,0,78,71,4,...,11,7,8,21,5,Connecticut,X,1,Texas,X
3,2003,146,1277,1400,25,N,0,76,54,8,...,12,1,2,28,7,Michigan St,X,1,Texas,X
4,2003,139,1345,1400,23,N,0,67,62,5,...,13,7,2,24,9,Purdue,X,1,Texas,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,2021,138,1382,1261,21,N,0,61,63,3,...,3,3,5,17,9,St Bonaventure,W,8,LSU,W
2358,2021,140,1276,1261,28,N,1,86,52,10,...,11,1,4,17,1,Michigan,W,8,LSU,W
2359,2021,138,1268,1163,22,N,1,63,43,9,...,10,4,4,14,10,Maryland,W,7,Connecticut,W
2360,2021,138,1325,1438,21,N,1,62,50,7,...,9,1,2,9,13,Ohio,X,4,Virginia,X


In [25]:
#
#tournament_df
#tournament_df.to_csv('Tourney_Data.csv', encoding='utf-8', index=False)

In [26]:
# Winning_Teams

winning_Teams = tournament_df[tournament_df['Won'] == 1]

#winning_Teams.to_csv('Tourney_Winning_Teams.csv', encoding='utf-8', index=False)



losing_Teams = tournament_df[tournament_df['Won'] == 0]

#losing_Teams.to_csv('Tourney_Losing_Teams.csv', encoding='utf-8', index=False)
losing_Teams